In [4]:
# DEscarga de datos de Worldpop por país

import requests
import pandas as pd
import os

# Sitio base
site_d = "https://hub.worldpop.org/rest/data/"
# Categorías
cat_d  = [ "age_structures", "births" ]
# Variables
var_d  = [ [ "wcba2015", "sapya1km" ], [ "bic" ] ]

# Variable a utilizar
i = 1
j = 0

# Guardamos la lista de países
URL_info = f"{site_d}{cat_d[i]}/{var_d[i][j]}"
df = pd.DataFrame(requests.get(URL_info).json()["data"])
countries = df["iso3"].values

# Iteramos para cada país
for c in countries:
    path_f = f"../temp/{var_d[i][j]}/{var_d[i][j]}_{c}.tif"
    # Solo si no se ha descargado ya
    if not os.path.exists(path_f):
        print(c)
        # Sitio web del archivo
        URL_file = pd.DataFrame( requests.get( f"{URL_info}?iso3={c}"
            ).json()["data"])["files"].iloc[-1][0]
        # Si no existe la carpeta la crea
        if not os.path.exists(f"../temp/{var_d[i][j]}/"):
            os.mkdir(f"../temp/{var_d[i][j]}/")
        # Guardamos el archivo
        with open(path_f, "wb") as f: f.write(requests.get(URL_file).content)

In [21]:
import os
import pandas as pd
import numpy as np
import xarray as xr

# Códigos nacionales
ix  = "ISO_A3"

# Tabla base
iso = "../../Bases_de_datos/Country_ISO_code.csv"
df_iso = pd.read_csv(iso).set_index("country-code")
df_iso.index.name = "ISO_N3"
df_iso = df_iso.rename({"alpha-3": ix}, axis = 1)

path = "../../Bases_de_datos/WorldPop/global_level0_1km_2000_2020"
ds = xr.open_dataset(f"{path}.tif").isel(band = 0).drop_vars(
    ["band", "spatial_ref"] ).rename_dims(
    {"x": "lon", "y": "lat"} ).rename_vars(
    {"x": "lon", "y": "lat"} )

for row in df_iso.itertuples():
    print(row.name, end = "                                   \r")
    ds_1 = ds.where(ds == row.Index, np.nan ).dropna(
        dim = "lat", how = "all" ).dropna( dim = "lon", how = "all" )
    if 0 in ( ds_1["lon"].shape[0], ds_1["lat"].shape[0] ):
        print("\n Not present in dataset")
    else:
        ds_2 = ds.sel( { "lon": slice(ds_1["lon"].values[0],
            ds_1["lon"].values[-1]),
            "lat": slice(ds_1["lat"].values[0],
            ds_1["lat"].values[-1]) } )
        ds_2 = ds_2.where( ds_2 == row.Index, np.nan )
        ds_2 = ds_2.where( ds_2.isnull(), 1 ).to_netcdf(
            f"{path}/{row.ISO_A3}.nc" )

Antarctica                                       
 Not present in dataset
Christmas Island                                                   
 Not present in dataset
Cocos (Keeling) Islands                                   
 Not present in dataset


In [17]:
c = 336
print(row.name, end = "                                   \r")
ds_1 = ds.where(ds == c, np.nan ).dropna(
    dim = "y", how = "all" ).dropna( dim = "x", how = "all" )
if 0 in ( ds_1["x"].shape[0], ds_1["y"].shape[0] ):
    print("\n Not present in dataset")
else:
    ds_2 = ds.sel( { "x": slice(ds_1["x"].values[0], ds_1["x"].values[-1]),
        "y": slice(ds_1["y"].values[0], ds_1["y"].values[-1]) } )
    ds_2 = ds_2.where( ds_2 == c, np.nan )
    ds_2 = ds_2.where( ds_2.isnull(), 1 )

In [20]:
ds_2.to_netcdf(
            f"{path}/VAT.nc" )

In [14]:
(  ds_1["x"].shape[0], ds_1["y"].shape[0] )

(0, 0)

In [5]:
ds_1["x"].shape[0]

0